<a href="https://colab.research.google.com/github/ezzeddinegasmi/DRL_comparative_study/blob/main/SAC_Vend_11_4_10H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install stable-baselines3[extra] pygame

In [25]:
 2.  L’environnement

SyntaxError: invalid character '’' (U+2019) (<ipython-input-25-aaed27e8197e>, line 1)

In [ ]:
import gym
from gym import spaces
import numpy as np
import random

class BreakoutContinuousEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(BreakoutContinuousEnv, self).__init__()
        self.screen_width = 400
        self.screen_height = 300
        self.paddle_width = 60
        self.paddle_height = 10
        self.ball_size = 8

        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

        self.observation_space = spaces.Box(
            low=np.array([0, 0, -5, -5, 0]),
            high=np.array([self.screen_width, self.screen_height, 5, 5, self.screen_width]),
            dtype=np.float32
        )
        self.reset()

    def reset(self):
        self.paddle_x = self.screen_width / 2
        self.ball_x = self.screen_width / 2
        self.ball_y = self.screen_height / 2
        self.ball_vx = random.choice([-3, 3])
        self.ball_vy = -3
        self.score = 0
        self.done = False
        return self._get_obs()

    def _get_obs(self):
        return np.array([self.ball_x, self.ball_y, self.ball_vx, self.ball_vy, self.paddle_x], dtype=np.float32)

    def step(self, action):
        move = float(action[0]) * 10
        self.paddle_x = np.clip(self.paddle_x + move, 0, self.screen_width - self.paddle_width)

        self.ball_x += self.ball_vx
        self.ball_y += self.ball_vy

        if self.ball_x <= 0 or self.ball_x >= self.screen_width:
            self.ball_vx *= -1
        if self.ball_y <= 0:
            self.ball_vy *= -1

        if (self.ball_y + self.ball_size >= self.screen_height - self.paddle_height) and \
           (self.paddle_x <= self.ball_x <= self.paddle_x + self.paddle_width):
            self.ball_vy *= -1
            self.score += 1
            reward = 1.0
        elif self.ball_y > self.screen_height:
            self.done = True
            reward = -10.0
        else:
            reward = -0.01

        return self._get_obs(), reward, self.done, {}

    def render(self, mode='human'):
        pass  # Pygame peut être ajouté ici si nécessaire

    def close(self):
        pass


In [ ]:
  Entraîner le modèle SAC (avec Monitor et logger les récompenses)

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
import os

# Créer le dossier de logs
log_dir = "./logs/"
os.makedirs(log_dir, exist_ok=True)

# Créer l’environnement + Monitor
env = BreakoutContinuousEnv()
env = Monitor(env, log_dir)

# Créer et entraîner le modèle
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)  # Tu peux augmenter à 50000+

In [ ]:
Le graphe des récompenses

In [ ]:
from stable_baselines3.common.results_plotter import load_results, ts2xy
import matplotlib.pyplot as plt

results = load_results(log_dir)
x, y = ts2xy(results, 'timesteps')

plt.figure(figsize=(12, 6))
plt.plot(x, y, label='Reward per episode')
plt.xlabel("Timesteps")
plt.ylabel("Reward")
plt.title("SAC Training Rewards (via Monitor)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
Sauvegarde

In [ ]:
model.save("sac_breakout_continuous")

# Rechargement
model = SAC.load("sac_breakout_continuous", env=env)

In [ ]:
Visualisation avec Pygame

In [ ]:
import pygame

def render(self, mode='human'):
    if not hasattr(self, 'screen'):
        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption("Breakout Continuous")
        self.clock = pygame.time.Clock()

    self.screen.fill((0, 0, 0))  # Fond noir

    # Dessiner la balle
    pygame.draw.circle(self.screen, (255, 255, 255), (int(self.ball_x), int(self.ball_y)), self.ball_size)

    # Dessiner la raquette
    pygame.draw.rect(
        self.screen,
        (0, 255, 0),
        pygame.Rect(int(self.paddle_x), self.screen_height - self.paddle_height, self.paddle_width, self.paddle_height)
    )

    pygame.display.flip()
    self.clock.tick(60)

In [ ]:
def close(self):
    if hasattr(self, 'screen'):
        pygame.quit()

In [ ]:
env = BreakoutContinuousEnv()  # Pas besoin de Monitor ici
model = SAC.load("sac_breakout_continuous", env=env)

obs = env.reset()
total_reward = 0

for step in range(300):  # ou plus
    env.render()  # Affiche le jeu
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        break

env.close()
print(f"Total reward: {total_reward}")
